In [188]:
import pandas as pd
from fuzzywuzzy import fuzz, process

In [249]:
# Carregar as bases de dados
df_cervejas = pd.read_csv('./datasets/raw/CERVEJAS.csv')
df_harmonizacoes = pd.read_csv('./datasets/preprocessed/HARMONIZACOES_preprocessed.csv')

In [251]:
df_harmonizacoes

,estilos_cervejas,pratos_pt,pratos
0,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...","{'Flooded (tortillas)', 'Empanadas', 'Tuna (pa..."
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor..."
2,amber lager,"{'lasanha', 'abacate (guacamole ou salada)', '...","{'Avocado (guacamole or salad)', 'Peru', 'Chic..."
3,american brown ale,"{'feijão/ feijoada', 'carne com chili', 'bife ...","{'Tamboril (fish)', 'Meat roll', 'Grilled lamb..."
4,german pilsner,"{'sopa (caldo de piranha)', 'caranguejo', 'emp...","{'Empanadas', 'Pizza Mussarela/Marguerita', 'C..."
...,...,...,...
62,vienna lager,"{'risoto de funghi', 'sopa minestrone'}","{'Sopa minestrone', 'Risato de mushroom'}"
63,traditional,{'tamboril (peixe)'},{'Tamboril (fish)'}
64,bock,{'tamboril (peixe)'},{'Tamboril (fish)'}
65,belgian dark strong,{'terrine'},{'Terrine'}


In [184]:
df_cervejas

,name,number,category,categorynumber,overallimpression,aroma,appearance,flavor,mouthfeel,comments,...,fgmax,abvmin,abvmax,srmmin,srmmax,commercialexamples,tags,entryinstructions,currentlydefinedtypes,strengthclassifications
0,Altbier,7B,Amber Bitter European Beer,7,"A moderately colored, well-attenuated, bitter ...",Malty and rich with grainy characteristics lik...,"The color ranges from amber to deep copper, st...","Malt profile similar to the aroma, with an ass...",Medium body. Smooth. Medium to medium-high car...,"Classic, traditional examples in the Altstadt ...",...,1.014,4.3,5.5,9.0,17.0,"Bolten Alt, Diebels Alt, Füchschen Alt, Origin...","standard-strength, amber-color, top-fermented,...",NaN,NaN,NaN
1,American Amber Ale,19A,Amber And Brown American Beer,19,"An amber, hoppy, moderate-strength American cr...",Low to moderate hop aroma reflective of Americ...,"Deep amber to coppery-brown in color, sometime...",Moderate to high hop flavor with similar chara...,Medium to medium-full body. Medium to high car...,Can overlap in color with darker American pale...,...,1.015,4.5,6.2,10.0,17.0,"Anderson Valley Boont Amber Ale, Bell's Amber ...","standard-strength, amber-color, top-fermented,...",NaN,NaN,NaN
2,American Barleywine,22C,Strong American Ale,22,"A very strong, malty, hoppy, bitter American a...",Strong malt and hop aroma dominates. Hops are ...,"Color ranges from amber to medium copper, rare...",Similar malt and hop flavors as the aroma (sam...,"Full-bodied and chewy, with a velvety, lusciou...","Sometimes labeled as ""Barley Wine"" or ""Barleyw...",...,1.030,8.0,12.0,9.0,18.0,"Anchor Old Foghorn, Bell's Third Coast Old Ale...","very-high-strength, amber-color, top-fermented...",NaN,NaN,NaN
3,American Brown Ale,19C,Amber And Brown American Beer,19,A malty but hoppy standard-strength American a...,Moderate malty-sweet to malty-rich aroma with ...,Light to very dark brown color. Clear. Low to ...,Medium to moderately-high malty-sweet or malty...,Medium to medium-full body. More bitter versio...,Most commercial American Browns are not as agg...,...,1.016,4.3,6.2,18.0,35.0,"Avery Ellie's Brown Ale, Big Sky Moose Drool B...","standard-strength, dark-color, top-fermented, ...",NaN,NaN,NaN
4,American IPA,21A,Ipa,21,"A decidedly hoppy and bitter, moderately stron...",A prominent to intense hop aroma often featuri...,Color ranging from medium gold to light reddis...,Medium to very high hop flavor (same descripto...,"Medium-light to medium body, with a smooth tex...","The basis for many modern variations, includin...",...,1.014,5.5,7.5,6.0,14.0,"Bell's Two-Hearted Ale, Cigar City Jai Alai, F...","high-strength, pale-color, top-fermented, nort...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Wee Heavy,17C,Strong British Ale,17,"Rich, sweet malt depth with caramel, toffee, a...","Strong bready-toasty malt, with a high caramel...","Light copper to dark brown color, often with d...","Rich, bready-toasty malt that is often full an...","Medium-full to full-bodied, sometimes with a t...",A range of strengths is allowable; not all ver...,...,1.040,6.5,10.0,14.0,25.0,"Belhaven Wee Heavy, Broughton Old Jock, McEwan...","high-strength, amber-color, top-fermented, bri...",NaN,NaN,NaN
92,Weissbier,10A,German Wheat Beer,10,"A pale, refreshing, lightly-hopped German whea...","Moderate to strong esters and phenols, typical...","Pale straw to gold in color. Very thick, mouss...",Low to moderately strong banana and clove flav...,Medium-light to medium body; never heavy. Fluf...,"Also known as hefeweizen or weizenbier, partic...",...,1.014,4.3,5.6,2.0,6.0,"Ayinger Bräuweisse, Distelhäuser Hell Weizen, ...","standard-strength, pale-color, top-fermented, ...",The entrant may specify whether the yeast shou...,NaN,NaN
93,Weizenbock,10C,German Wheat Beer,10,A strong and malty German wheat beer combining...,Medium-high to high malty richness with a sign...,"Very thick, moussy, long-lasting head. C

In [253]:
df_harmonizacoes = df_harmonizacoes.rename(columns={'estilos_cervejas':'estilos_cervejas_brejas'})

In [255]:
# Garantir que as colunas certas estão sendo usadas
coluna_cerveja = 'name'
coluna_harmonizacao = 'estilos_cervejas_brejas'


In [257]:
# Remover as linhas com valores nulos ou strings vazias na coluna de harmonizações
df_harmonizacoes = df_harmonizacoes.dropna(subset=[coluna_harmonizacao])
df_harmonizacoes = df_harmonizacoes[df_harmonizacoes[coluna_harmonizacao].str.strip() != '']

In [259]:
# Converter os valores para string, se necessário
df_harmonizacoes[coluna_harmonizacao] = df_harmonizacoes[coluna_harmonizacao].astype(str)
df_cervejas[coluna_cerveja] = df_cervejas[coluna_cerveja].astype(str)

In [120]:
# # Lista para armazenar as linhas com correspondências duplicadas
# novas_linhas = []

# # Iterar sobre cada linha da base de harmonização
# for index, row in df_harmonizacoes.iterrows():
#     estilo = row[coluna_harmonizacao]
    
#     # Encontrar as correspondências mais similares com fuzzywuzzy
#     cervejas_encontradas = process.extract(estilo, df_cervejas[coluna_cerveja], limit=5, scorer=fuzz.token_sort_ratio)
    
#     # Filtrar por correspondências com pontuação alta (ajustar o limite conforme necessário)
#     cervejas_similares = [match[0] for match in cervejas_encontradas if match[1] >= 82]  # Similaridade >= 85
    
#     if cervejas_similares:
#         # Adicionar a linha com todas as cervejas encontradas
#         for cerveja in cervejas_similares:
#             nova_linha = row.copy()
#             nova_linha['estilos_cervejas'] = cerveja
#             novas_linhas.append(nova_linha)
#     else:
#         # Se nenhuma cerveja similar for encontrada, adicionar a linha original com estilo vazio ou o nome original
#         nova_linha = row.copy()
#         nova_linha['estilos_cervejas'] = estilo  # Ou "" se preferir deixar em branco
#         novas_linhas.append(nova_linha)

# # Criar o novo DataFrame com todas as linhas (duplicadas e sem correspondências)
# df_novo_harmonizacao = pd.DataFrame(novas_linhas)


In [261]:
# Lista para armazenar as linhas com correspondências únicas
novas_linhas = []

# Iterar sobre cada linha da base de harmonização
for index, row in df_harmonizacoes.iterrows():
    estilo = row[coluna_harmonizacao]
    
    # Buscar correspondências similares na coluna 'estilos_cervejas'
    cervejas_encontradas = process.extract(estilo, df_cervejas[coluna_cerveja], limit=5, scorer=fuzz.token_sort_ratio)
    
    # Filtrar por correspondências com pontuação alta
    cervejas_similares = [match for match in cervejas_encontradas if match[1] >= 85]
    
    # Se não encontrar correspondências suficientes, buscar na coluna 'category'
    if not cervejas_similares:
        categorias_encontradas = process.extract(estilo, df_cervejas['category'], limit=5, scorer=fuzz.token_sort_ratio)
        categorias_similares = [match for match in categorias_encontradas if match[1] >= 85]
        
        # Iterar sobre cada categoria similar encontrada
        for categoria_similar in categorias_similares:
            categoria = categoria_similar[0]
            
            # Encontrar todos os estilos de cerveja para essa categoria
            estilos_correspondentes = df_cervejas[df_cervejas['category'] == categoria][coluna_cerveja].values
            
            # Adicionar uma linha para cada estilo correspondente
            for estilo_cerveja in estilos_correspondentes:
                nova_linha = row.copy()
                nova_linha['estilos_cervejas'] = estilo_cerveja
                nova_linha['category'] = categoria
                
                # Verificar se a combinação já está na lista para evitar duplicações
                if not any((linha['estilos_cervejas'] == estilo_cerveja and linha['category'] == categoria) for linha in novas_linhas):
                    novas_linhas.append(nova_linha)
    else:
        # Adicionar as correspondências de 'estilos_cervejas' diretamente
        for match in cervejas_similares:
            estilo_cerveja = match[0]
            categoria = df_cervejas.loc[df_cervejas[coluna_cerveja] == estilo_cerveja, 'category'].values[0]

            # Criar o dicionário com os dados da nova linha
            nova_linha = row.copy()
            nova_linha['estilos_cervejas'] = estilo_cerveja
            nova_linha['category'] = categoria

            # Verificar se a combinação já está na lista para evitar duplicações
            if not any((linha['estilos_cervejas'] == estilo_cerveja and linha['category'] == categoria) for linha in novas_linhas):
                novas_linhas.append(nova_linha)

# Criar o novo DataFrame com as linhas exclusivas
df_novo_harmonizacao = pd.DataFrame(novas_linhas)


In [177]:
# Lista para armazenar as linhas com correspondências duplicadas
novas_linhas = []

# Iterar sobre cada linha da base de harmonização
for index, row in df_harmonizacoes.iterrows():
    estilo = row[coluna_harmonizacao]
    
    # Encontrar as correspondências mais similares com fuzzywuzzy
    cervejas_encontradas = process.extract(estilo, df_cervejas[coluna_cerveja], limit=5, scorer=fuzz.token_sort_ratio)
    
    # Filtrar por correspondências com pontuação alta (ajustar o limite conforme necessário)
    cervejas_similares = [match[0] for match in cervejas_encontradas if match[1] >= 85]  # Similaridade >= 80
    
    # Adicionar a linha com todas as cervejas encontradas
    for cerveja in cervejas_similares:
        nova_linha = row.copy()
        nova_linha['estilos_cervejas'] = cerveja
        novas_linhas.append(nova_linha)


In [276]:
df_novo_harmonizacao

,estilos_cervejas_brejas,pratos_pt,pratos,estilos_cervejas,category
0,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...","{'Flooded (tortillas)', 'Empanadas', 'Tuna (pa...",American Pale Ale,Pale American Ale
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",American IPA,Ipa
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",Hazy IPA,Ipa
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",Specialty IPA,Ipa
3,american brown ale,"{'feijão/ feijoada', 'carne com chili', 'bife ...","{'Tamboril (fish)', 'Meat roll', 'Grilled lamb...",American Brown Ale,Amber And Brown American Beer
3,american brown ale,"{'feijão/ feijoada', 'carne com chili', 'bife ...","{'Tamboril (fish)', 'Meat roll', 'Grilled lamb...",American Strong Ale,Strong American Ale
4,german pilsner,"{'sopa (caldo de piranha)', 'caranguejo', 'emp...","{'Empanadas', 'Pizza Mussarela/Marguerita', 'C...",German Pils,Pale Bitter European Beer
7,sweet stout,"{'sorvete', 'torta de framboesa', 'torta de ma...","{'Ice cream', 'Panna cotta', 'Banana and Briga...",Sweet Stout,Dark British Beer
9,flanders red ale,"{'ostras', 'foie gras', 'sardinha', 'arenque (...","{'Foie gras', 'Oysters', 'LAKE (GORDIOUS FISH)...",Flanders Red Ale,European Sour Ale
13,berliner weisse,{'arenque (peixe gorduroso)'},{'LAKE (GORDIOUS FISH)'},Berliner Weisse,European Sour Ale


In [287]:
# Contagem de linhas por categoria (verificação) e ordenação
row_count_per_category = df_novo_harmonizacao['category'].value_counts().sort_values(ascending=False)

# Contagem de estilos únicos por categoria (como antes) e ordenação
# unique_styles_per_category = df_novo_harmonizacao.groupby('category')['estilos_cervejas'].nunique().sort_values(ascending=False)
row_count_per_estilo = df_novo_harmonizacao['estilos_cervejas'].value_counts().sort_values(ascending=False)


# Exibir os resultados
print("Número total de linhas por categoria (ordenado do maior para o menor):\n", row_count_per_category)
print("\nNúmero de estilos de cervejas únicos por categoria (ordenado do maior para o menor):\n", row_count_per_estilo)


Número total de linhas por categoria (ordenado do maior para o menor):
 category
Monastic Ale                     3
Strong Belgian Ale               3
Ipa                              3
Belgian Ale                      3
Amber And Brown American Beer    2
Pale Bitter European Beer        2
Dark British Beer                2
European Sour Ale                2
Dark European Lager              2
Amber Bitter European Beer       2
Strong European Beer             1
Strong American Ale              1
Pale Malty European Lager        1
Irish Beer                       1
Pale American Ale                1
Fruit Beer                       1
Strong British Ale               1
Amber Malty European Lager       1
Standard American Beer           1
American Porter And Stout        1
Name: count, dtype: int64

Número de estilos de cervejas únicos por categoria (ordenado do maior para o menor):
 estilos_cervejas
American Pale Ale            1
Berliner Weisse              1
American Amber Ale         

In [293]:
contagem = df_novo_harmonizacao.groupby(['estilos_cervejas_brejas', 'pratos_pt', 'pratos']).size().reset_index(name='count')
contagem_filtrada = contagem[contagem['count'] > 1]

print(contagem_filtrada)


   estilos_cervejas_brejas                                          pratos_pt  \
2       american brown ale  {'feijão/ feijoada', 'carne com chili', 'bife ...   
18                     ipa  {'chouriço', 'enchiladas (tortilhas)', 'carne ...   

                                               pratos  count  
2   {'Tamboril (fish)', 'Meat roll', 'Grilled lamb...      2  
18  {'Avocado (guacamole or salad)', 'Flooded (tor...      3  


In [158]:
df_novo_harmonizacao = pd.DataFrame(novas_linhas)

In [265]:
df_novo_harmonizacao

,estilos_cervejas_brejas,pratos_pt,pratos,estilos_cervejas,category
0,american pale ale,"{'atum (massas ou sanduíches)', 'enchiladas (t...","{'Flooded (tortillas)', 'Empanadas', 'Tuna (pa...",American Pale Ale,Pale American Ale
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",American IPA,Ipa
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",Hazy IPA,Ipa
1,ipa,"{'chouriço', 'enchiladas (tortilhas)', 'carne ...","{'Avocado (guacamole or salad)', 'Flooded (tor...",Specialty IPA,Ipa
3,american brown ale,"{'feijão/ feijoada', 'carne com chili', 'bife ...","{'Tamboril (fish)', 'Meat roll', 'Grilled lamb...",American Brown Ale,Amber And Brown American Beer
3,american brown ale,"{'feijão/ feijoada', 'carne com chili', 'bife ...","{'Tamboril (fish)', 'Meat roll', 'Grilled lamb...",American Strong Ale,Strong American Ale
4,german pilsner,"{'sopa (caldo de piranha)', 'caranguejo', 'emp...","{'Empanadas', 'Pizza Mussarela/Marguerita', 'C...",German Pils,Pale Bitter European Beer
7,sweet stout,"{'sorvete', 'torta de framboesa', 'torta de ma...","{'Ice cream', 'Panna cotta', 'Banana and Briga...",Sweet Stout,Dark British Beer
9,flanders red ale,"{'ostras', 'foie gras', 'sardinha', 'arenque (...","{'Foie gras', 'Oysters', 'LAKE (GORDIOUS FISH)...",Flanders Red Ale,European Sour Ale
13,berliner weisse,{'arenque (peixe gorduroso)'},{'LAKE (GORDIOUS FISH)'},Berliner Weisse,European Sour Ale


In [267]:
# Salvar o novo DataFrame
df_novo_harmonizacao.to_csv('./datasets/preprocessed/HARMONIZACOES_processed.csv', index=False)

